In [1]:
#!/usr/bin/env python
# coding: utf-8

# # Image Compression 

# ## Import Libraries

# In[1]:


import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import sys
import cv2
from sklearn.feature_extraction import image
from skimage.util.shape import view_as_blocks
import time
from datetime import timedelta
import math
import os
from collections import Counter
import matplotlib.pyplot as plt

# ## Function Definitions

# In[2]:


def distance(patch,cluster,C):
    min = sys.float_info.max
    for i in range(C):
        temp = np.square(patch - cluster[i]).mean()
        if(temp < min):
            min = temp
            idx = i
    return idx


# In[3]:


def distortion(patches,cluster,index,prow,row,col):
    mse = np.zeros(prow)
    for i in range(prow):
        mse[i] = np.sum(np.square(patches[i] - cluster[index[i]]))
    return np.sum(mse)/(row*col)


# In[4]:


def centroid(patches,index,prow,pcol,C):
    num_per_cluster = np.zeros(C)
    code_vector = np.zeros((C,p*p))
    for i in range(prow):
        num_per_cluster[index[i]] += 1
        for j in range(pcol):
            code_vector[index[i]][j] += patches[i][j]
    for i in range(C):
            code_vector[i] /= num_per_cluster[i]
    return np.rint(code_vector)


# In[5]:


def training(patches,cluster,index,C,prow,row,col,k,r):
    temp = sys.maxsize
    for iteration in range(100):
        for i in range(prow):
            index[i] = distance(patches[i],cluster,C)
        dist[k] = distortion(patches,cluster,index,prow,row,col)
        print("Distortion " + str(iteration) + " (r = " + str(r[k]) + ") : " + str(dist[k]))
        cluster = centroid(patches,index,prow,pcol,C)
        if((temp - dist[k]) < 0.5):
            break
        temp = dist[k]
    return dist


# In[6]:


def partition(img,block_shape,N,p):
    N = int(N)
    p = int(p)
    view = view_as_blocks(np.ascontiguousarray(img), block_shape)
    patches = np.reshape(view,(p*N,p))
    patches = np.reshape(view,(N,p*p))
    return patches


# In[7]:


def stich(patches,cluster,index,row,col,N,p,prow):
    N = int(N)
    p = int(p)
    for i in range(prow):
        patches[i] = cluster[index[i]]
    z = int(row*col/p)
    patches = np.reshape(patches,(z,p))
    patches = np.reshape(patches,(int(row/p),int(col/p),p,p))
    patches = np.swapaxes(patches,1,2)
    patches = np.reshape(patches,(p,p,N))
    patches = np.reshape(patches,(row,col))
    return patches

# In[8]:

def show_img(img,compressed_image,r,figure):
    cv2.imshow('Compressed Image{}_ R = {}'.format(figure,r),compressed_image)
    cv2.imwrite(r'C:\Users\sport\Google Drive\First Semester\Topics in DS\Projects\Compressed_Image\compressed_image{}_R{}.jpg'.format(figure,r), compressed_image)



# In[9]:


def clip_image(img):
    row,col = img.shape
    row = 2**int(math.log(row,2))
    col = 2**int(math.log(col,2))
    return img[0:int(row),0:int(col)],int(row),int(col)


# In[10]:


def test(cluster,C,p,block_shape,r):
    #path = os.path.join(os.path.expanduser('~'), 'Users', 'sport','Google Drive','First Semester','Topics in DS', 'Projects', 'lena-gray.bmp')
    img1 = cv2.imread(r'C:\Users\sport\Google Drive\First Semester\Topics in DS\Projects\lena-gray.bmp',0)
    img,row,col = clip_image(img1)
    cv2.imshow('Original Image Testing',img)
    N = int(row*col/(p*p))
    print(N,p,row,col)
    patches = partition(img,block_shape,N,p)
    prow,pcol = patches.shape
    index = np.zeros(prow).astype(int)
    for i in range(prow):
        index[i] = distance(patches[i],cluster,C)
    print('Test Image Distortion for R={}:  '.format(r), distortion(patches,cluster,index,prow,row,col))
    reconstructed_image = stich(patches,cluster,index,row,col,N,p,prow) 
    show_img(img,reconstructed_image,r,1)

# def entropy_coding():

# ## Main Code Begins Here

# In[11]:

#path = os.path.join(os.path.expanduser('~'), 'Users', 'sport','Google Drive','First Semester','Topics in DS', 'Projects', 'wolves.png')
#C:\Users\sport\Google Drive\First Semester\Topics in DS\Projects   
img1 = cv2.imread(r'C:\Users\sport\Google Drive\First Semester\Topics in DS\Projects\wolves.png',0)
#print(img1)
img,row,col = clip_image(img1)
cv2.imshow('Original Image Training',img)
p = 4
if (p == 2):
    r = [0.2, 0.4, 0.6, 0.8, 1.0]
else:
    r = [0.05, 0.1, 0.15, 0.2, 0.25]
start_time = np.zeros(len(r))
dist = np.zeros(len(r))
elapsed_time = np.zeros(len(r))
R = np.zeros(len(r))

for k in range(len(r)):
    C = int(round(2**(r[k]*(p**2))))
    N = row*col/(p*p)
    start_time[k] = time.time()
    print("\n")
    print('No. of Clusters: ',C)

    block_shape = (p,p)
    patches = partition(img,block_shape,N,p)
    prow,pcol = patches.shape
    
    cluster = np.zeros((C,p*p))
    index = np.zeros(prow).astype(int)
    for j in range(C):
        cluster[j] = [patches[np.random.randint(prow)][i] for i in range(pcol)]

    dist = training(patches,cluster,index,C,prow,row,col,k,r)
    if k == len(r)-1:
        plt.plot(r, dist)
        plt.show()
    compressed_image = stich(patches,cluster,index,row,col,N,p,prow) 

    # print("Cluster : ")
    # print(cluster)
    # print("Patches : ")
    # print(patches)
    # print("Index : ")
    # print(index)

    unique, counts = np.unique(index, return_counts=True) 
    dict(zip(unique, counts))
    # print(Counter(index))
    # print(unique, counts)
    Ni = np.zeros(len(counts))
    for q in range(len(counts)):
        Ni[q] = counts[q]
    pi = np.zeros(len(cluster))
    H = np.zeros(len(cluster))
    # print("N : " + str(N))
    pi = Ni/N
    # print("Probability : ")
    # print(pi)
    H = -np.sum(np.multiply(pi, np.log2(pi)))
    print("H : " + str(H))
    R[k] = H/(p**2)
    print("Rate = " + str(R[k]))
    rate_change = (r[k]-R[k])/r[k]
    if rate_change > 0:
        print("There is a " + str(rate_change) + "% reduction in coding rate")
    if rate_change < 0:
        print("There is a " + str(-rate_change) + "% increase in coding rate")

    show_img(img,compressed_image,r[k],0)

    test(cluster,C,p,block_shape,r[k])

    elapsed_time[k] = time.time() - start_time[k]
    print("Duration: {} s ".format(elapsed_time[k]))
print('Done')
cv2.waitKey(0)
cv2.destroyAllWindows()





No. of Clusters:  2
Distortion 0 (r = 0.05) : 9273.006784439087
Distortion 1 (r = 0.05) : 4257.309766769409
Distortion 2 (r = 0.05) : 2118.636484146118
Distortion 3 (r = 0.05) : 2001.2147388458252
Distortion 4 (r = 0.05) : 1967.8938236236572
Distortion 5 (r = 0.05) : 1956.1888599395752
Distortion 6 (r = 0.05) : 1952.3361377716064
Distortion 7 (r = 0.05) : 1951.106559753418
Distortion 8 (r = 0.05) : 1950.6849613189697
H : 0.9414482425526826
Rate = 0.05884051515954266
There is a 0.17681030319085314% increase in coding rate
16384 4 512 512
Test Image Distortion for R=0.05:   6776.694957733154
Duration: 7.935237646102905 s 


No. of Clusters:  3
Distortion 0 (r = 0.1) : 7662.648138046265
Distortion 1 (r = 0.1) : 1734.224557876587
Distortion 2 (r = 0.1) : 1377.3158893585205
Distortion 3 (r = 0.1) : 1309.9619789123535
Distortion 4 (r = 0.1) : 1296.0950241088867
Distortion 5 (r = 0.1) : 1291.1432781219482
Distortion 6 (r = 0.1) : 1288.418004989624
Distortion 7 (r = 0.1) : 1286.9936046600342

<Figure size 640x480 with 1 Axes>

H : 3.731028812049359
Rate = 0.23318930075308494
There is a 0.06724279698766022% reduction in coding rate
16384 4 512 512
Test Image Distortion for R=0.25:   4313.382270812988
Duration: 65.49066829681396 s 
Done
